In [1]:
from mm import *

In [2]:
from sklearn import *
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Charubaiel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

# pos \ neg - комменты с твиттера http://study.mokoron.com/
# labeled каггловский датасет по токсикам https://www.kaggle.com/blackmoon/russian-language-toxic-comments

In [4]:
twitter_neg = pd.read_csv('data/negative.csv',sep=';',header=None,usecols=[3])
twitter_pos = pd.read_csv('data/positive.csv',sep=';',header=None,usecols=[3])
vk_all = pd.read_csv('data/labeled.csv')

In [5]:
ttl_toxic = vk_all.append(twitter_neg.rename(columns={3:'comment'})).fillna(1)
df = ttl_toxic.append(twitter_pos.rename(columns={3:'comment'})).fillna(0)


In [6]:
df['toxic'].value_counts(normalize=True)

0.0    0.516058
1.0    0.483942
Name: toxic, dtype: float64

In [7]:
scores = {}

### stupid baseline

In [8]:
tf = feature_extraction.text.TfidfVectorizer(min_df=10)

In [9]:
tf.fit(vk_all['comment'])

TfidfVectorizer(min_df=10)

In [10]:
logreg = linear_model.LogisticRegression()
pipe = pipeline.make_pipeline(tf,logreg)

In [11]:
scores['baseline'] = pd.DataFrame(model_selection.cross_validate(pipe,vk_all['comment'],vk_all['toxic'],scoring=['f1','precision','recall'],cv=5)).mean()

In [12]:
pd.DataFrame(scores)

,baseline
fit_time,0.378491
score_time,0.069566
test_f1,0.694865
test_precision,0.872140
test_recall,0.594260


### micro EDA

In [13]:
eda = vk_all.copy()

In [14]:
eda['txt_len'] = eda['comment'].str.len()
eda['txt_len_avg'] = eda['comment'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
eda['txt_words'] = eda['comment'].str.count(' ')
eda['txt_puncts'] = eda['comment'].str.count('[^\w^ ]')
eda['txt_upper_cnt'] = eda['comment'].apply(lambda x: len([i for i in x if i.isupper()]))
eda['txt_pos_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\)|D',text)))
eda['txt_neg_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))

In [15]:
eda.groupby('toxic')[eda.filter(regex='txt').columns].agg(['mean','std'])

txt_len             txt_len_avg            txt_words             \
             mean         std        mean       std       mean        std   
toxic                                                                       
0.0    194.213332  274.750067    5.312308  1.063207  29.713436  40.811415   
1.0    141.392665  261.776417    5.467866  1.466984  21.449233  42.106635   

      txt_puncts           txt_upper_cnt            txt_pos_punc            \
            mean       std          mean        std         mean       std   
toxic                                                                        
0.0     7.481849  9.135183      3.984978   7.812980     0.279261  0.709825   
1.0     5.972234  9.188326      6.595939  26.932055     0.094488  0.434023   

      txt_neg_punc            
              mean       std  
toxic                         
0.0       0.499478  1.353467  
1.0       0.454414  1.689128

In [16]:
eda.iloc[:,1:].corr()

,toxic,txt_len,txt_len_avg,txt_words,txt_puncts,txt_upper_cnt,txt_pos_punc,txt_neg_punc
toxic,1.000000,-0.091782,0.060394,-0.094138,-0.077608,0.072997,-0.136895,-0.014424
txt_len,-0.091782,1.000000,0.024703,0.991756,0.924170,0.370052,0.382291,0.529863
txt_len_avg,0.060394,0.024703,1.000000,-0.025150,0.016616,0.022158,-0.001665,0.037176
txt_words,-0.094138,0.991756,-0.025150,1.000000,0.929034,0.368631,0.378389,0.514993
txt_puncts,-0.077608,0.924170,0.016616,0.929034,1.000000,0.346096,0.418016,0.497885
txt_upper_cnt,0.072997,0.370052,0.022158,0.368631,0.346096,1.000000,0.115176,0.692265
txt_pos_punc,-0.136895,0.382291,-0.001665,0.378389,0.418016,0.115176,1.000000,0.395420
txt_neg_punc,-0.014424,0.529863,0.037176,0.514993,0.497885,0.692265,0.395420,1.000000


### work with text

In [17]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
stopwords = nltk.corpus.stopwords.words('russian')

In [18]:
vk_all['no_punct'] = vk_all['comment'].apply(lambda x: re.sub('\W',' ',x).replace('  ',' ').strip())

In [19]:
def normalizer (text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    clean_text = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if token.lemma not in stopwords:
            clean_text.append(token.lemma)
    return ' '.join(clean_text)

In [20]:
text = '|||'.join(vk_all['no_punct'])

In [21]:
new_text = normalizer(text)

In [22]:
vk_all['normalize_text'] = new_text.split('| | |')

In [23]:
scores['normalize'] = pd.DataFrame(model_selection.cross_validate(pipe,vk_all['normalize_text'],vk_all['toxic'],scoring=['f1','precision','recall'],cv=5)).mean()

In [24]:
pd.DataFrame(scores)

,baseline,normalize
fit_time,0.378491,0.325110
score_time,0.069566,0.051616
test_f1,0.694865,0.725498
test_precision,0.872140,0.894935
test_recall,0.594260,0.622248


### features

In [25]:
def features_from_text(df):
    df = pd.DataFrame(df)
    df.columns=['text']
    df['txt_len'] = df['text'].str.len()
    df['txt_len_avg'] = df['text'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
    df['txt_words'] = df['text'].str.count(' ')
    df['txt_puncts'] = df['text'].str.count('[^\w^ ]')
    df['txt_upper_cnt'] = df['text'].apply(lambda x: len([i for i in x if i.isupper()]))
    df['txt_pos_punc'] = df['text'].apply(lambda text: len(re.findall(r'\)|D',text)))
    df['txt_neg_punc'] = df['text'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))
    
    return df.drop('text',axis=1).fillna(0)

In [26]:
new_fe=preprocessing.FunctionTransformer(features_from_text)
fu = pipeline.FeatureUnion([('idf',tf),('fe',new_fe)])

In [27]:
clf = linear_model.LogisticRegression()
pipe_fe = pipeline.Pipeline([('fu',fu),('clf',clf)])

In [28]:
scores['features'] = pd.DataFrame(model_selection.cross_validate(pipe_fe,vk_all['normalize_text'],vk_all['toxic'],scoring=['f1','precision','recall'],cv=5)).mean()

In [29]:
pd.DataFrame(scores)

,baseline,normalize,features
fit_time,0.378491,0.325110,0.735161
score_time,0.069566,0.051616,0.173052
test_f1,0.694865,0.725498,0.706075
test_precision,0.872140,0.894935,0.887883
test_recall,0.594260,0.622248,0.597176


### vectors

In [30]:
from navec import Navec

In [31]:
nav = Navec.load('emb_navec.tar')

In [32]:
def get_sentence_vecror(sentence):
    sent_vec = []
    for i in sentence.split():
        if i in nav:
            sent_vec.append(nav[i])
        else:
            sent_vec.append(nav['<unk>'])
    if sentence.strip() == '':
        sent_vec = [nav['<unk>']]
    return np.mean(sent_vec,axis=0)

In [33]:
vk_vectors = np.vstack(vk_all['normalize_text'].apply(get_sentence_vecror))

In [34]:
scores['sample_vec'] = pd.DataFrame(model_selection.cross_validate(logreg,vk_vectors,vk_all['toxic'],scoring=['f1','precision','recall'],cv=5)).mean()

In [35]:
pd.DataFrame(scores)

,baseline,normalize,features,sample_vec
fit_time,0.378491,0.325110,0.735161,0.416257
score_time,0.069566,0.051616,0.173052,0.011397
test_f1,0.694865,0.725498,0.706075,0.795245
test_precision,0.872140,0.894935,0.887883,0.844346
test_recall,0.594260,0.622248,0.597176,0.759430


### combo

In [36]:
word_vectorizer = feature_extraction.text.TfidfVectorizer(min_df=10)
word_vectorizer.fit(vk_all['normalize_text'])

TfidfVectorizer(min_df=10)

In [37]:
char_vectorizer = feature_extraction.text.TfidfVectorizer(
    sublinear_tf=True,
    analyzer='char',
    ngram_range=(2,5))
char_vectorizer.fit(vk_all['normalize_text'])

TfidfVectorizer(analyzer='char', ngram_range=(2, 5), sublinear_tf=True)

In [38]:
idf_fu = pipeline.FeatureUnion([('idf_w',word_vectorizer),('idf_c',char_vectorizer)])

In [39]:
clf = linear_model.LogisticRegression()
pipe_idf_fe = pipeline.Pipeline([('fu',idf_fu),('clf',clf)])

In [40]:
scores['idf_features'] = pd.DataFrame(model_selection.cross_validate(pipe_idf_fe,vk_all['normalize_text'],vk_all['toxic'],scoring=['f1','precision','recall'],cv=5)).mean()

In [41]:
pd.DataFrame(scores)

,baseline,normalize,features,sample_vec,idf_features
fit_time,0.378491,0.325110,0.735161,0.416257,9.148327
score_time,0.069566,0.051616,0.173052,0.011397,1.040702
test_f1,0.694865,0.725498,0.706075,0.795245,0.797823
test_precision,0.872140,0.894935,0.887883,0.844346,0.905753
test_recall,0.594260,0.622248,0.597176,0.759430,0.725239


### final